# Bot Classificador - Telegram

In [ ]:
!sudo apt update && sudo apt install ffmpeg -y
!pip install openai-whisper
!pip install telebot
!pip install openai-whisper
!pip install ultralytics


###Bibliotecas usadas:

In [2]:
import telebot
import io
import tensorflow as tf
import tensorflow_hub as hub
import librosa
import whisper
import csv
import zipfile
import requests
import matplotlib.pyplot as plt
import glob
import random
import time
from ultralytics import YOLO
from google.colab.patches import cv2_imshow
import numpy as np # Added for image processing
import cv2

### Classificação do áudio:

In [3]:
class ClassificadorAudio:
    def __init__(self):
        print("Carregando YAMNet...")
        self.model = hub.load('https://tfhub.dev/google/yamnet/1')
        class_map_path = self.model.class_map_path().numpy().decode('utf-8')
        self.class_names = self._ler_labels(class_map_path)

    def _ler_labels(self, path):
        classes = []
        with tf.io.gfile.GFile(path) as f:
            reader = csv.DictReader(f)
            for row in reader:
                classes.append(row['display_name'])
        return classes

    def identificar_som(self, arquivo_memoria):
        arquivo_memoria.seek(0)
        audio, sr = librosa.load(arquivo_memoria, sr=16000, mono=True)
        scores, _, _ = self.model(audio)
        media_scores = tf.reduce_mean(scores, axis=0)
        idx_max = tf.argmax(media_scores)
        return self.class_names[idx_max], media_scores[idx_max].numpy()


###Transcrição de áudio:

In [4]:
class TranscritorWhisper:
    def __init__(self):
        modelo = input("Modelo do Whisper (large/base):")
        print(f"Carregando Whisper (modelo {modelo})...")
        self.model = whisper.load_model(modelo)

    def transcrever(self, arquivo_memoria):
        arquivo_memoria.seek(0)
        audio, sr = librosa.load(arquivo_memoria, sr=16000)
        resultado = self.model.transcribe(audio=audio, fp16=False)
        arquivo_memoria.close()
        return resultado['text']

### Classificação da imagem:

In [5]:
class ClassificadorImagem:
    def __init__(self):
        print("Carregando YOLOv8n.pt...")
        self.model = YOLO("yolov8n.pt")

    def classificar_imagem(self, arquivo_memoria):
        # Check if input is BytesIO or path string
        if isinstance(arquivo_memoria, io.BytesIO):
            file_bytes = np.asarray(bytearray(arquivo_memoria.read()), dtype=np.uint8)
            image = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)
        elif isinstance(arquivo_memoria, str):
            image = cv2.imread(arquivo_memoria)
        else:
            return "Erro: Formato de entrada de imagem não suportado.", ""


        if image is None:
              return "Erro ao carregar a imagem.", "" # Return two values for consistency

        # Está executando o yolo
        result = self.model(image)

        if len(result[0].boxes) == 0:
              return image, "Nenhum objeto detectado." # Return image and caption

        # Nomes das classes já presentes no yolo
        name_class = result[0].names

        caption_parts = []
        # Esse loop passa por cada parte do yolo já treinasdo e mostra o resultado da análise
        for box in result[0].boxes:
          class_id = int(box.cls)
          confidence = float(box.conf) * 100

          # Desenvolvendo o comportamento da bounding box
          x1, y1, x2, y2 = map(int, box.xyxy[0])   # bounding box

          # desenha a bounding box (vermelha)
          cv2.rectangle(
            image,
              (x1, y1),
              (x2, y2),
              (0, 0, 255), # vermelho (BGR)
              2
          )
          # Desenha o texto na imagem
          cv2.putText(
            image,
              f"{name_class[class_id]} {confidence:.1f}%",
              (x1, max(y1 - 30, 40)),
              cv2.FONT_HERSHEY_SIMPLEX,
              0.6,
              (0, 0, 255),
              2
          )

          # Imprime no prompt
          detected_info = (
              f"Classe: {name_class[class_id]} | "
              f"Confiança: {confidence:.2f}% | "
              f"Box: ({x1}, {y1}, {x2}, {y2})"
          )
          caption_parts.append(f"{name_class[class_id]} ({confidence:.1f}%)")

        legenda = "Objetos detectados: " + ", ".join(caption_parts) if caption_parts else "Nenhum objeto detectado."
        return image, legenda

### Base do Bot

In [6]:
class BotTelegram:
    def __init__(self, token, **kwargs): # Accept kwargs for cooperative inheritance
        self.__token = token  # Atributo privado
        self.bot = telebot.TeleBot(self.__token)
        # No super().__init__(**kwargs) here, as object (its ultimate base) doesn't take kwargs.
    def _configurar_handlers(self):

        @self.bot.message_handler(commands=['start'])
        def start(message):
            nome = message.from_user.first_name
            self.bot.reply_to(message, f"Olá, {nome}! Sou um BOT criado para transcrever seus áudios e classificar áudios e imagens.")

        @self.bot.message_handler(commands=['ajuda'])
        def ajuda(message):
            self.bot.reply_to(message, "Experimente me enviar uma imagem ou áudio e eu vou tentar classificá-lo.")

        @self.bot.message_handler(content_types=['video', 'sticker', 'animation', 'video_note', 'document', 'poll'])
        def responder(message):
            self.bot.reply_to(message, "Desculpe, não trabalhamos com esse tipo de arquivo. Tente nos enviar uma imagem ou um áudio.")

        @self.bot.message_handler(content_types=['text'])
        def texto(message):
            self.bot.reply_to(message, "Desculpe, não consigo entender o que você diz. Não sou adaptado para compreender e responder textos. Eu apenas consigo classificar e/ou transcrever imagens e áudios que você me enviar.")

    def executar(self):
        print("Bot Online...")
        self.bot.infinity_polling()

###Bot para o áudio:

In [7]:
class BotAudio(BotTelegram):
    def __init__(self, token, motor_som, motor_texto, **kwargs):
        super().__init__(token=token, **kwargs) # Pass token as keyword argument

        self.classificador_audio = motor_som
        self.transcritor = motor_texto
        # Handlers are configured by calling this method (which calls super()._configurar_handlers())

    def _configurar_handlers_audio(self):
        super()._configurar_handlers()
        @self.bot.message_handler(content_types=['voice', 'audio'])
        def tratar_audio(message):
            status = self.bot.reply_to(message, "⏳ Analisando áudio...")
            try:
                file_info = self.bot.get_file(message.voice.file_id if message.voice else message.audio.file_id)
                downloaded_file = self.bot.download_file(file_info.file_path)
                audio_ram = io.BytesIO(downloaded_file)

                # Reset stream position before classification and transcription
                audio_ram.seek(0)
                classe, confianca = self.classificador_audio.identificar_som(audio_ram)
                audio_ram.seek(0) # Reset again for transcription

                resposta = f"✅ Som identificado: {classe} ({confianca:.2%})\n"

                if "Speech" in classe or "Conversation" in classe:
                    self.bot.edit_message_text(resposta + "🎙️ Transcrevendo...", message.chat.id,
                                               status.message_id)
                    texto = self.transcritor.transcrever(audio_ram)
                    resposta += f"\n📝 Transcrição: {texto}"

                self.bot.edit_message_text(resposta, message.chat.id, status.message_id, parse_mode="Markdown")

            except Exception as e:
                resposta += f"\nErro: {str(e)}"
                self.bot.edit_message_text(resposta, message.chat.id, status.message_id, parse_mode="Markdown")

            finally:
                if 'audio_ram' in locals() and not audio_ram.closed:
                    audio_ram.close()

###Bot para a imagem:

In [8]:
class BotImagem(BotTelegram):
    def __init__(self, token, motor_imagem, **kwargs):
        super().__init__(token=token, **kwargs) # Pass token as keyword argument
        self.classificador_imagem = motor_imagem

    def _configurar_handlers_image(self):
        super()._configurar_handlers() # Call parent's general handlers

        @self.bot.message_handler(content_types=['photo'])
        def tratar_foto(message):
            status = self.bot.reply_to(message, "⏳ Imagem recebida. Vamos tentar classificá-la.\n\nBaixando imagem...")

            file_id = message.photo[-1].file_id # Pega a maior resolução
            file_info = self.bot.get_file(file_id)
            downloaded_file = self.bot.download_file(file_info.file_path)

            imagem_ram = io.BytesIO(downloaded_file)

            self.bot.edit_message_text("Download concluído.\n\nClassificando imagem...", message.chat.id, status.message_id)

            imagem_resultante, legenda = self.classificador_imagem.classificar_imagem(imagem_ram)
            imagem_ram.close()

            self.bot.delete_message(message.chat.id, status.message_id)

            if isinstance(imagem_resultante, str): # Error message from classifier
                self.bot.send_message(message.chat.id, imagem_resultante)
            else:
                # Convert numpy array to bytes for sending
                success, encoded_image = cv2.imencode('.png', imagem_resultante)
                if success:
                    photo_bytes = io.BytesIO(encoded_image.tobytes())
                    self.bot.send_photo(message.chat.id, photo_bytes, caption=legenda)
                    photo_bytes.close()
                else:
                    self.bot.send_message(message.chat.id, "Erro ao converter imagem classificada para envio.")

##BotAdmin

In [9]:
class BotAdmin(BotAudio, BotImagem):
    def __init__(self, token, motor_som, motor_texto, motor_imagem, **kwargs):
        # Pass all arguments as keyword arguments to super(), which handles MRO.
        super().__init__(token=token, motor_som=motor_som, motor_texto=motor_texto, motor_imagem=motor_imagem, **kwargs)
        # After all parents are initialized, configure all handlers through the MRO.
        super()._configurar_handlers_image()
        super()._configurar_handlers_audio()
        super()._configurar_handlers()

##Execução:

In [ ]:
if __name__ == "__main__":
    TOKEN = '7812768588:AAFyWirqLGt_B-cdMip7sWMp8n_bVBYvzyo'

    # 1. Instancia as IAs primeiro
    ia_som = ClassificadorAudio()
    ia_texto = TranscritorWhisper()
    ia_imagem = ClassificadorImagem()

    # 2. Instancia a classe filha, que já configura a classe mãe internamente
    # Pass all arguments as keyword arguments
    meu_bot = BotAdmin(token=TOKEN, motor_som=ia_som, motor_texto=ia_texto, motor_imagem=ia_imagem)

    # 3. Inicia o polling
    meu_bot.executar()

Modelo Whisper: Adicionei a opção de escolha entre os modelos large e base para o operador que irá iniciar o bot. O modelo large é muito preciso, mas é lento e pesado. O google colab suporta ele no limite da memória RAM disponível e um bug ou recompilação reinicia a sessão. O modelo base é inferior, mas é significativamente mais leve e veloz.

Falta organizar melhor as interações do Bot com relação às mensagens do usuário, como quando envia uma mensagem aleatória, ou um arquivo não compatível. Além disso, as dinâmicas de edição de mensagem do TeleBot tornam a interação mais dinâmica, mas seria bom dar uma revisada para ficar visualmente mais agradável em tudo.

Falta fazer a documentação também.